<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4 --break-system-packages
!brew install python-requests
!pip3 install pandas --break-system-packages

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
cyctl        fern-api     libpeas@1    openjdk@21   toipe        vexctl
==> New Casks
alcom               clion@eap           get-api             proton-pass

You have 18 outdated formulae and 2 outdated casks installed.

To reinstall 2.31.0, run:
  brew reinstall python-requests


In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests
test1 = requests.get(static_url)
if test1.ok:
    print("Request was successful")  

Request was successful


In [10]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

soup = BeautifulSoup(test1.text, "html.parser")
soup_data.title



<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [18]:
#soup_data.head
page_title = soup_data.title.string
print(page_title)



List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [21]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables =  soup_data.find_all('table')
#html_tables


Starting from the third table is our target table contains the actual launch records.


In [23]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [24]:
# Apply find_all() function with `th` element on first_launch_table
html_tables =  soup_data.find_all('table')
first_launch_table = first_launch_table.find_all('th')
print(first_launch_table)


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell.
    Input: a table data cell element from which extra rows are extracted.
    """
    # Extract unwanted tags
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    # Join contents and strip leading/trailing whitespace
    colunm_name = ' '.join(row.contents).strip()

    # Validate it's not empty and not just digits
    if colunm_name and not colunm_name.isdigit():
        return colunm_name
    else:
        return None  # Return None if it's invalid or empty



[<th scope="col">Flight No.
</th>, <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>, <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>, <th scope="col">Launch site
</th>, <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>, <th scope="col">Payload mass
</th>, <th scope="col">Orbit
</th>, <th scope="col">Customer
</th>, <th scope="col">Launch<br/>outcome
</th>, <th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th>, <th rowspan="2" scope="row" style="text-align:center;">1
</th>, <th rowspan="2" scope="row" style="text-align:center;">2
</th>, <th rowspan="2" scope="ro

In [25]:
#print(range(len(first_launch_table)-1))
cols=[]
#print(cols)

for i in (range(len(first_launch_table)-1)):
    #print(i)
    cols.append(extract_column_from_header(first_launch_table[i]))

print(cols)


['Flight No.', 'Date and time ( )', None, 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', None, None, None, None, None, None, None]


In [26]:

# Use list comprehension to filter non-empty and non-None values
column_names = [name for name in cols if name is not None and len(name.strip()) > 0]

print(column_names)  # Output: ['Name', 'Age', 'Occupation']

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [27]:


# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [28]:
print(column_names)



['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [29]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']
#del launch_dict['Customer ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
#launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


In [30]:
# First, import the necessary library
import pandas as pd

# Initialize variables
extracted_row = 0
temp = []

# Iterate over the tables
for table_number, table in enumerate(soup_data.find_all('table', "wikitable plainrowheaders collapsible")):
    # Iterate over the rows in the table
    for rows in table.find_all("tr"):
        flag = False
        
        # Check if row has a number as first element
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        
        if flag:
            # Create a new dictionary for each valid row
            launch_dict = {}

            # Increment the counter for extracted rows
            extracted_row += 1

            # Flight Number
            launch_dict["Flight No."] = flight_number
            
            # Table cells in the row
            row = rows.find_all("td")

            # Extract data and populate the dictionary
            launch_dict["Launch Site"] = row[2].a.string
            launch_dict["Payload"] = row[3].a.string
            launch_dict["Payload mass"] = get_mass(row[4])

            datatimelist = date_time(row[0])
            launch_dict["Date"] = datatimelist[0].strip(',')
            launch_dict["Time"] = datatimelist[1]

            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict["Version Booster"] = bv
            
            launch_dict["Orbit"] = row[5].a.string

            # Customer data is optional, so we need to ensure it exists
            if len(row) > 6 and row[6].a:
                launch_dict["Customer"] = row[6].a.string

            # Launch outcome
            launch_dict["Launch outcome"] = list(row[7].strings)[0]

            # Booster landing
            launch_dict["Booster landing"] = landing_status(row[8])

            # Append the dictionary to the list of extracted rows
            temp.append(launch_dict)

# Now create the data frame from the list of dictionaries
launch_df = pd.DataFrame(temp)

# Display the data frame
print(launch_df)

launch_df.to_csv('spacex_web_scraped.csv', index=False)

    Flight No. Launch Site                               Payload Payload mass  \
0            1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1            2       CCAFS                                Dragon            0   
2            3       CCAFS                                Dragon       525 kg   
3            4       CCAFS                          SpaceX CRS-1     4,700 kg   
4            5       CCAFS                          SpaceX CRS-2     4,877 kg   
..         ...         ...                                   ...          ...   
116        117       CCSFS                              Starlink    15,600 kg   
117        118         KSC                              Starlink   ~14,000 kg   
118        119       CCSFS                              Starlink    15,600 kg   
119        120         KSC                         SpaceX CRS-22     3,328 kg   
120        121       CCSFS                                 SXM-8     7,000 kg   

                Date   Time

In [31]:
extracted_row = 0
temp = []

#Extract each table 
for table_number,table in enumerate(soup_data.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            # Create a new dictionary for each valid row
            launch_dict = {}
            extracted_row += 1
            
            
            # Flight Number value
            #TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict["Flight No."] = flight_number
            #print(launch_dict)
            
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict["Launch site"] = launch_site
            #print(launch_dict["Launch site"])
            #print(launch_site)
            
            #print(launch_dict)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict["Payload"] = payload
            #print(launch_dict["Payload"])
            
            #print(launch_dict)
            
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"] = payload_mass
            #print(payload)
            
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict["Date"] = date
            #date = { "": date,}
            #launch_dict.append(date)
            #print(date)
            #print(launch_dict)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            
            time = datatimelist[1]
            launch_dict["Time"] = time
            #time = { "": time,}
            #launch_dict.append(time)
            
            #print(time)
            #print(launch_dict["Time"])
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            
            launch_dict["Version Booster"] = bv
            #print(bv)
            

            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict["Orbit"] = orbit
            
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict["Launch outcome"] = launch_outcome
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #launch_dict["Booster landing"] = booster_landing
            #print(booster_landing)
            
            #print(launch_dict)
            temp.append(launch_dict)

           
         

           
#print(temp)

# Now create the data frame from the list of dictionaries
launch_df = pd.DataFrame(temp)

# Display the data frame
print(launch_df)

    Flight No. Launch site                               Payload Payload mass  \
0            1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1            2       CCAFS                                Dragon            0   
2            3       CCAFS                                Dragon       525 kg   
3            4       CCAFS                          SpaceX CRS-1     4,700 kg   
4            5       CCAFS                          SpaceX CRS-2     4,877 kg   
..         ...         ...                                   ...          ...   
116        117       CCSFS                              Starlink    15,600 kg   
117        118         KSC                              Starlink   ~14,000 kg   
118        119       CCSFS                              Starlink    15,600 kg   
119        120         KSC                         SpaceX CRS-22     3,328 kg   
120        121       CCSFS                                 SXM-8     7,000 kg   

                Date   Time

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [35]:
launch_df.columns

Index(['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Date', 'Time',
       'Version Booster', 'Orbit', 'Launch outcome'],
      dtype='object')

In [32]:
print(launch_dict)


{'Flight No.': '121', 'Launch site': 'CCSFS', 'Payload': 'SXM-8', 'Payload mass': '7,000 kg', 'Date': '6 June 2021', 'Time': '04:26', 'Version Booster': 'F9 B5', 'Orbit': 'GTO', 'Launch outcome': 'Success\n'}


In [33]:

#df = pd.DataFrame(launch_dict)
unique_items = list(set(launch_dict))
#print(launch_dict) 
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
print(df['Time'])
#print(df)



0    04:26
Name: Time, dtype: object


In [34]:
df.to_csv('spacex_web_scraped.csv', index=False)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
